In [1]:
import barzur20aft
import compiler
import numpy as np
import pandas

In [2]:
def bitcoin_mdp(*args, **kwargs):
    model = barzur20aft.Bitcoin(*args, **kwargs)
    c = compiler.Compiler(model)
    return c.mdp()

In [3]:
alpha = [0.01, 0.05, 0.1, 0.2, 0.3, 0.35, 0.4, 0.45, 0.49]
ptmdp = []
mdp = []
for a in alpha:
    print(a)
    m = bitcoin_mdp(alpha=a, gamma=1, maximum_fork_length=25)
    mdp.append(m)
    ptmdp.append(barzur20aft.ptmdp(m, horizon=100))

0.01
0.05
0.1
0.2
0.3
0.35
0.4
0.45
0.49


In [4]:
vi = []
for i, m in enumerate(ptmdp):
    res = m.value_iteration(value_eps=0.01)
    vi.append(res)
    v = res["vi_value"]
    p = res["vi_policy"]
    iter = res["vi_iter"]
    print(
        f"alpha={alpha[i]:.2f} iter={iter:3d} v[0]={v[0]:.2f} p={p[:10]} {sum(p)} {hash(p.tobytes()) % 10000}"
    )

alpha=0.01 iter= 30 v[0]=0.14 p=[0 0 1 0 3 0 1 0 0 0] 1046 7900
alpha=0.05 iter=195 v[0]=3.23 p=[0 0 1 0 3 0 1 0 0 0] 949 582
alpha=0.10 iter=351 v[0]=9.04 p=[0 0 1 0 3 0 1 0 0 0] 861 2012
alpha=0.20 iter=534 v[0]=22.87 p=[0 0 1 0 3 0 1 0 0 0] 771 752
alpha=0.30 iter=664 v[0]=40.86 p=[0 0 1 0 3 0 1 0 0 0] 782 5126
alpha=0.35 iter=718 v[0]=52.17 p=[0 0 1 0 3 0 1 0 0 0] 773 7542
alpha=0.40 iter=763 v[0]=65.48 p=[0 0 1 0 3 0 1 0 0 0] 778 9087
alpha=0.45 iter=795 v[0]=80.41 p=[0 0 1 0 3 0 1 0 0 0] 733 5928
alpha=0.49 iter=804 v[0]=91.34 p=[0 0 1 0 3 0 1 0 0 0] 660 88


In [5]:
def rows():
    for i, a in enumerate(alpha):
        rpp = mdp[i].reward_per_progress(vi[i]["vi_policy"], eps=0.0001)
        yield dict(alpha=a) | rpp


pandas.DataFrame(rows())

,alpha,ss_time,rpp,rpp_iter,rpp_time
0,0.01,0.787586,0.010100,2,0.015358
1,0.05,0.732544,0.052632,2,0.015727
2,0.10,0.725546,0.111111,2,0.016007
3,0.20,0.740875,0.250000,2,0.016495
4,0.30,0.791383,0.428544,2,0.016623
5,0.35,0.717292,0.538146,2,0.016599
6,0.40,0.847136,0.664888,2,0.017119
7,0.45,0.990060,0.809704,2,0.016677
8,0.49,1.936042,0.919795,2,0.033739
